In [278]:
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
import requests
from urllib.request import urlopen
import numpy as np
from collections import defaultdict

In [289]:
# def geturl(url):

#     response = requests.get(url)
#     page= response.text
#     soup = BeautifulSoup(page, 'html.parser')    
#     all_matches = soup.find_all('div', class_ = 'reviews')
    
#     reviewer_type = soup.find_all('span', attrs = {'itemprop': 'name'})
#     reviewer = soup.find_all('span', attrs = {'itemprop': 'author'})   
#     date = soup.find_all('span', attrs = {'itemprop': 'datePublished'})    
#     review = soup.find_all('span', attrs = {'itemprop': 'description'})    
#     star = soup.find_all('span', attrs = {'itemprop': 'ratingValue'})
#     star = star[1:]    
#     location = soup.find('div', attrs={'class': 'locality'}).text if soup.find('div', attrs={'class': 'locality'}) else ""      
#     name_of_senior_living = soup.find('div', attrs={'class': 'name'}).text if  soup.find('div', attrs={'class': 'name'}) else ""    
#     caring_type = soup.find('p', attrs = {'class': 'offers'}).text if soup.find('p', attrs = {'class': 'offers'}) else ""

#     dic = {'reviewer_type': reviewer_type,
#            'reviewer_name': reviewer,
#            'date': date,
#            'star': star,
#            'review':review,
#            'location': location,
#            'name_of_senior_living': name_of_senior_living,
#            'caring_type': caring_type
#           }
        
    
#     return dic


def geturl(url):

    response = requests.get(url)
    page= response.text
    soup = BeautifulSoup(page, 'html.parser')    
    all_matches = soup.find_all('div', class_ = 'reviews')
    
    reviewer_type = soup.find_all('span', attrs = {'itemprop': 'name'})    
    reviewer = soup.find_all('span', attrs = {'itemprop': 'author'})   
    date = soup.find_all('span', attrs = {'itemprop': 'datePublished'})    
    review = soup.find_all('span', attrs = {'itemprop': 'description'})    
    star = soup.find_all('span', attrs = {'itemprop': 'ratingValue'})
    if len(star) == len(review) + 1:
        star = star[1:]
    
    dataframe = pd.DataFrame(
          {'reviewer_type': reviewer_type,
           'reviewer_name': reviewer,
           'date': date,
           'star': star,
           'review':review   
          })
    
    location = soup.find('div', attrs={'class': 'locality'}).text if soup.find('div', attrs={'class': 'locality'}) else "" 
    dataframe['location'] = location
    
    name_of_senior_living = soup.find('div', attrs={'class': 'name'}).text if soup.find('div', attrs={'class': 'name'}) else ""
    dataframe['name_of_senior_living'] = name_of_senior_living
    
    caring_type = soup.find('p', attrs = {'class': 'offers'}).text if soup.find('p', attrs = {'class': 'offers'}) else ""
    dataframe['type'] = caring_type
    
    return dataframe

def clean_bracket(dataset):
    
    dataset['reviewer_type'] = dataset['reviewer_type'].apply(str)
    dataset['reviewer_type'] = dataset['reviewer_type'].apply(lambda x: re.sub(r'\<span itemprop="name"\>', "", x))
    dataset['reviewer_type'] = dataset['reviewer_type'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    dataset['reviewer_name'] = dataset['reviewer_name'].apply(str)
    dataset['reviewer_name'] = dataset['reviewer_name'].apply(lambda x: re.sub(r'\<span itemprop="author"\>', "", x))
    dataset['reviewer_name'] = dataset['reviewer_name'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    dataset['date'] = dataset['date'].apply(str)
    dataset['date'] = dataset['date'].apply(lambda x: re.sub(r'\<span itemprop="datePublished"\>', "", x))
    dataset['date'] = dataset['date'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    dataset['star'] = dataset['star'].apply(str)
    dataset['star'] = dataset['star'].apply(lambda x: re.sub(r'\<span itemprop="ratingValue"\>', "", x))
    dataset['star'] = dataset['star'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    dataset['review'] = dataset['review'].apply(str)
    dataset['review'] = dataset['review'].apply(lambda x: re.sub(r'\<span itemprop="description"\>', "", x))
    dataset['review'] = dataset['review'].apply(lambda x: re.sub(r'\<\/span\>', "", x))
    
    return dataset

In [281]:
#get all california cities urls
url = "https://www.caring.com/senior-living/assisted-living/california"
response = requests.get(url)
page = urlopen(url)
soup = BeautifulSoup(page, 'lxml')

#get url 
urls = []    
data = soup.findAll(attrs={'id': 'cities'})
for div in data:
    links = div.findAll('a')
    for a in links:
        urls.append(a['href'])
      


In [291]:
#get all review dat(a from ca cities      
data = []

for url in urls:
    response = requests.get(url)
    page = urlopen(url)
    soup = BeautifulSoup(page, 'lxml')
    all_matches = soup.findAll(attrs = {'class':['search-result']})

    for reviews in all_matches:
        review_url = reviews['data-link']
        df = geturl(review_url) 

        if not df.empty:
            data.append(df)



In [293]:
ca_data = pd.concat(data)
ca_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90565 entries, 0 to 0
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   reviewer_type          90565 non-null  object
 1   reviewer_name          90565 non-null  object
 2   date                   90565 non-null  object
 3   star                   90565 non-null  object
 4   review                 90565 non-null  object
 5   location               90565 non-null  object
 6   name_of_senior_living  90565 non-null  object
 7   type                   90565 non-null  object
dtypes: object(8)
memory usage: 6.2+ MB


In [248]:
ca_data.to_csv('')

['https://www.caring.com/senior-living/assisted-living/california/agoura-hills',
 'https://www.caring.com/senior-living/assisted-living/california/alameda',
 'https://www.caring.com/senior-living/assisted-living/california/alamo',
 'https://www.caring.com/senior-living/assisted-living/california/albany',
 'https://www.caring.com/senior-living/assisted-living/california/alhambra',
 'https://www.caring.com/senior-living/assisted-living/california/aliso-viejo',
 'https://www.caring.com/senior-living/assisted-living/california/alta-loma',
 'https://www.caring.com/senior-living/assisted-living/california/altadena',
 'https://www.caring.com/senior-living/assisted-living/california/american-canyon',
 'https://www.caring.com/senior-living/assisted-living/california/anaheim',
 'https://www.caring.com/senior-living/assisted-living/california/angels-camp',
 'https://www.caring.com/senior-living/assisted-living/california/antelope',
 'https://www.caring.com/senior-living/assisted-living/california